### Scorecard Automation

#### General Outline

- First find **offset**
- Check date
- Read tables one by one
- check for missing tables
- Check data in each tabel
- Check for missing data
- Check if charts exist
- Validate data with rnramd

___

**Write function to determine which sheet contains which data**


#### List of offsets

> Network Summary Heading : 3

> Bing O&O Core Browse:

---

**Make these global vars so that they can be changed anytime**



In [1]:
x_offsets = [0, 21]

y_offsets = [6, 6]
widths = [7, 7]

heights = [10, 10]

#### Setting Development mode

In [4]:
import platform

print (platform.system())

dev_mode = platform.system()

if dev_mode == "Linux":
    root_path = "/media/shaunak/New Volume/my_projects/data science updated/python for data science/Scorecard Automate/data/"
elif dev_mode == "Windows":
    root_path = "D:/data_science-master/python for data science/Scorecard Automate/data/"


Linux


In [5]:
import openpyxl
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from matplotlib import style

import numpy as np

import quandl
import datetime

wb = openpyxl.load_workbook(filename=root_path + 'data.xlsx', 
                   read_only=True)
sheet_nw_sc = wb['Sheet4']

data_rows = []
for row in sheet_nw_sc['W10':'AC18']:
    data_cols = []
    for cell in row:
        data_cols.append(cell.value)
    data_rows.append(data_cols)

    
nw_sc_df = pd.DataFrame(data_rows)

nw_sc_df.set_index(0, inplace=True)

print (nw_sc_df)

"""
for row in sheet_nw_sc.iter_rows(min_row=21,min_col=23, max_col=30, max_row=25):
    for cell in row:
        print cell.value
"""

             1       2       3     4       5       6
0                                                   
oRPM     72.81 -0.0228 -0.0254  None  0.1085  0.1562
oPPC     92.5¢ -0.0106 -0.0194  None  0.0233  0.1022
oCY      0.079 -0.0123 -0.0061  None  0.0833  0.0490
oMLIY     0.56 -0.0251 -0.0093  None -0.0892 -0.1155
oVol     117mm -0.0219  0.0063  None  0.0200  0.0417
oRev   $8.52mm -0.0442 -0.0193  None  0.1306  0.2045
Rev    $8.55mm -0.0442 -0.0190  None  0.1327  0.1995
Vol      129mm -0.0437 -0.0092  None  0.0632  0.0795
RPM      66.52 -0.0005 -0.0099  None  0.0651  0.1111


'\nfor row in sheet_nw_sc.iter_rows(min_row=21,min_col=23, max_col=30, max_row=25):\n    for cell in row:\n        print cell.value\n'

#### Read in the file + determining offset

In [8]:
offset = 0

base_cell = 'W3'

network_sc_date_as_date = ""

workbook = openpyxl.load_workbook(filename = root_path + 'data3.xlsx', 
                   read_only=True)

network_sc_sheet = workbook['Sheet4']

first_heading = str(network_sc_sheet['W3'].value)

print (first_heading.strip('-'))


if first_heading is None or first_heading.split('-')[0].strip() != 'SEARCH BG SCORECARD':
    # print first_heading.split('-')[0].strip()
    print ("offset to be changed")
    
    first_heading = str(network_sc_sheet['W14'].value)
    
    if first_heading.split('-')[0].strip() == 'SEARCH BG SCORECARD':
        offset = 11
        print ("Offset changed")

None
offset to be changed
Offset changed


#### Offset changed at this step -- Move to separate function

#### Validating the date

In [9]:
def validate_date():
    global offset
    global base_cell
    global network_sc_sheet
    global network_sc_date_as_date
    # base cell = W3
    base_cell_num = int(base_cell[1:len(base_cell)])
    base_cell_num += offset
    
    cell_ref = base_cell[0] + str(base_cell_num)
    
    print (cell_ref)
    
    # Modifying the base cell
    
    base_cell = cell_ref
    
    first_heading = network_sc_sheet[cell_ref].value
    
    try:
        network_sc_date = first_heading.split('-')[1]
        network_sc_date = network_sc_date.strip()
        
        print (network_sc_date)
        
        network_sc_date_as_date = datetime.datetime.strptime(network_sc_date, "%B %d, %Y").date()
        
        # print network_sc_date_as_date
        
        current_date = datetime.date.today()
        
        # print datetime.date.today()
        
        # print network_sc_date_as_date + datetime.timedelta(days = 33) == datetime.date.today()
        
        days_lag = abs(current_date - network_sc_date_as_date).days
        
        if days_lag == 0:
            print ("Dates Validated!")
            return True
        else:
            print ("Scorecard is behind by: ", days_lag, "days")
            return False
        
    except IndexError:
        print ("There was some problem parsing the date")
        return False
    
validate_date()

W14
January 3, 2018
Scorecard is behind by:  37 days


False

#### Utility Fuction to get cell reference

In [10]:

def get_cell_ref(increment):
    global base_cell
    base_cell
    base_cell_int = int(base_cell[1:len(base_cell)])
    base_cell_int += increment
    new_cell_ref = base_cell[0] + str(base_cell_int)
    return new_cell_ref

get_cell_ref(3)

'W17'

#### Utility function to read in cells

In [11]:
def read_data(cell_ref):
    data = network_sc_sheet[cell_ref].value
    return data

#### Utility function to traverse cells

In [12]:
list_of_cols = []

for col in range(ord('A'), ord('Z')+1):
    list_of_cols.append(chr(col))

for col in range(ord('A'), ord('Z')+1):
    list_of_cols.append('A' + chr(col))
    
for col in range(ord('A'), ord('Z')+1):
    list_of_cols.append('B' + chr(col))
    
for col in range(ord('A'), ord('Z')+1):
    list_of_cols.append('C' + chr(col))
    
def traverse_cells(cell, incx, incy, mode=0):
    if mode == 0:
        # w3
        column_number = cell[0]
        row_number = int(cell[1:len(cell)])
    elif mode == 1:
        #ab1
        column_number = cell[0:2]
        row_number = int(cell[2:len(cell)])
        
    index_of_col = list_of_cols.index(column_number)
    index_of_col += incx
    new_col = list_of_cols[index_of_col]
    row_number += incy
    new_cell_ref = new_col + str(row_number)
    
    return new_cell_ref
    
    
    
traverse_cells('BZ1', 5, 28, 1)
    

'CE29'

#### Utility function to get a set of cells and return a pandas DF

In [13]:
def get_dataframe(start_cell, end_cell):
    data_rows = []
    for row in network_sc_sheet[start_cell: end_cell]:
        data_cols = []
        for cell in row:
            data_cols.append(cell.value)
        data_rows.append(data_cols)
    coverted_df = pd.DataFrame(data_rows) 
    return coverted_df

get_dataframe('W20', 'AC29')

,0,1,2,3,4,5,6
0,None,2018-01-02 00:00:00,Δ01/01,Δ12/26,None,Δ2017,Δ2016
1,oRPM,81.58,0.0853,-0.041,None,0.1006,0.1278
2,oPPC,93.4¢,0.1635,0.0898,None,0.0412,0.076
3,oCY,0.087,-0.0672,-0.1201,None,0.0571,-0.2513
4,oMLIY,0.65,-0.0183,-0.0679,None,-0.0495,-0.0467
5,oVol,104mm,0.5926,0.2321,None,-0.028,0.3034
6,oRev,$8.51mm,0.7284,0.1816,None,0.0698,0.47
7,Rev,$8.53mm,0.7278,0.1809,None,0.07,0.4624
8,Vol,117mm,0.5902,0.2339,None,-0.0032,0.3307
9,RPM,72.95,0.0866,-0.043,None,0.0724,0.0981


#### Utility function to extract numeric value from character string

In [14]:
def convert_to_numeric(number_str):
    list_of_chars = list(str(number_str))
    
    first_num_index = -1
    
    was_int = 0
    
    last_num_index = len(list_of_chars)-1
    for x in range(0, len(list_of_chars)):
        try:
            char_as_num = int(list_of_chars[x])
            was_int = 1
        except ValueError:
            was_int = 0
        if was_int == 1:
            break
            
    first_num_index = x
    # print first_num_index
    
    was_char = 1
    
    for x in range(last_num_index, -1, -1):
        try:
            char_as_num = int(list_of_chars[x])
            was_char = 0
        except ValueError:
            last_num_index -= 1
            was_char = 1
        if was_char == 0:
            break
    last_num_index = x
    # print last_num_index
    
    return float("".join(list_of_chars[first_num_index: last_num_index+1]))
    
convert_to_numeric('82.33')    

82.33

#### Base cell ref changed at this step -- Move to separate function

#### Dates validated

Check for Network Summary

#### Utility function to check missing and zero values in a list

In [15]:
import numbers

def check_missing_and_zero_values(column_list):
    for value in column_list:
            if isinstance(value, numbers.Number) is False:
                if value is None:
                    print ("Missing values in First Table")
                    return False
                else:
                    try:
                        length = len(value)
                        if length < 2:
                            print ("Missing values in First Table")
                            return False
                    except:
                        print ("Unknown stuff in First Table")
                        return False
            else:
                # number
                number_as_float = float(value)
                if float(value) == 0.0:
                    print ("Zero values in First Table")
                    return False
                
    return True
                    


#### Function to check 1st table data

> Make it resuable

In [17]:

def check_first_table_data(first_table_df):
    
    # maybe date check should be done here?
    print (first_table_df)
    
    # first_table_df[3][1] = 0
    # first_table_df[2][3] = '*'
    # first_table_df[2][4] = None
    
    if first_table_df.shape == (10, 7):
        
        # check for missing data
        
        first_table_df.drop([4], axis = 1, inplace = True)
        # print first_table_df.columns
        
        for column_index in first_table_df.columns[1:]:
            column_values = first_table_df[column_index][1:].tolist()
            if check_missing_and_zero_values(column_values) is False:
                return False
    

        
        print ("First Table data ok")
        return True
        
    else:
        print ("Dimension of First Table does not match")
        return False




In [22]:
def organic_metrics_validation(first_table_df):
    
    if first_table_df.shape == (10, 6):
        
        required_column = first_table_df[1][1:].tolist()
        print (required_column)
        
        organic_metrics = required_column[:6]
        non_organic_metrics = required_column[6:]
        
        oRev = organic_metrics[5]
        oVol = organic_metrics[4]
        Rev = non_organic_metrics[0]
        Vol = non_organic_metrics[1]
        
        print (oRev, oVol, Rev, Vol)
        
        # test case: oRev = 10.00
        
        oRev = convert_to_numeric(oRev)
        oVol = convert_to_numeric(oVol)
        Rev = convert_to_numeric(Rev)
        Vol = convert_to_numeric(Vol)
        
        print (oRev, oVol, Rev, Vol)
        
        if Rev < oRev or Vol < oVol:
            print ("Organic metrics greater than combined metrics which is not possible")
            return False
        
        return True
        
    else:
        print ("Dimension of First Table does not match")
        return False




#### Function for Network Summary First table

In [23]:
def network_summary_first_table():
    cell_ref = get_cell_ref(3)
    first_heading = str(read_data(cell_ref))
    # print first_heading
    
    if first_heading.strip() != 'NETWORK SUMMARY':
        print ("Cannot find heading: NETWORK SUMMARY")
        return False
    else:
        # all ok until now
        # try to find second heading
        
        second_heading = str(read_data(get_cell_ref(5)))
        if second_heading.strip() != 'Bing O&O Core Browse':
            print ("Cannot find heading: Bing O&O Core Browse")
            return False
        else:
            # read in the range of cells
            start_cell = get_cell_ref(6)
            # print start_cell
            
            end_cell = traverse_cells(start_cell, widths[0]-1, heights[0]-1, 0)
            # print end_cell
            
            network_summary_first_table_df = get_dataframe(start_cell, end_cell)
            
            
            network_summary_first_table_date = network_summary_first_table_df.loc[0][1]
            network_summary_first_table_date = datetime.datetime.date(network_summary_first_table_date)
            
            # print network_sc_date_as_date
            
            # print network_summary_first_table_date
            
            if (network_sc_date_as_date - network_summary_first_table_date).days == 1:
                print ("Dates validated in First Table")
                
                # check table data
                if check_first_table_data(network_summary_first_table_df) is False:
                    return False
                
                if organic_metrics_validation(network_summary_first_table_df) is False:
                    return False
            
            else:
                print ("Dates not validated in First Table")
                return False
            
            print ("All ok in First Table!!")
            
            return True
    

network_summary_first_table()

Dates validated in First Table
       0                    1       2       3     4       5       6
0   None  2018-01-02 00:00:00  Δ01/01  Δ12/26  None   Δ2017   Δ2016
1   oRPM                81.58  0.0853  -0.041  None  0.1006  0.1278
2   oPPC                93.4¢  0.1635  0.0898  None  0.0412   0.076
3    oCY                0.087 -0.0672 -0.1201  None  0.0571 -0.2513
4  oMLIY                 0.65 -0.0183 -0.0679  None -0.0495 -0.0467
5   oVol                104mm  0.5926  0.2321  None  -0.028  0.3034
6   oRev              $8.51mm  0.7284  0.1816  None  0.0698    0.47
7    Rev              $8.53mm  0.7278  0.1809  None    0.07  0.4624
8    Vol                117mm  0.5902  0.2339  None -0.0032  0.3307
9    RPM                72.95  0.0866  -0.043  None  0.0724  0.0981
First Table data ok
[81.58, '93.4¢', 0.087, 0.65, '104mm', '$8.51mm', '$8.53mm', '117mm', 72.95]
$8.51mm 104mm $8.53mm 117mm
8.51 104.0 8.53 117.0
All ok in First Table!!


True

#### Check Nw Summary Second Table

In [25]:
def network_summary_second_table():
    
    # read in the range of cells
    start_cell = traverse_cells(base_cell, x_offsets[1], y_offsets[1], 0)
    
    end_cell = traverse_cells(base_cell, x_offsets[1] + widths[1] - 1, y_offsets[1] + heights[1] - 1, 0)

    network_summary_second_table_df = get_dataframe(start_cell, end_cell)

    
    print (network_summary_second_table_df)

    network_summary_second_table_date = network_summary_second_table_df.loc[0][1]
    network_summary_second_table_date = datetime.datetime.date(network_summary_second_table_date)

    # print network_sc_date_as_date

    # print network_summary_first_table_date

    if (network_sc_date_as_date - network_summary_second_table_date).days == 1:
        print ("Dates validated in Second Table")

        # check table data
        if check_first_table_data(network_summary_second_table_df) is False:
            return False

    else:
        print ("Dates not validated in Second Table")
        return False

    print ("All ok in Second Table!!")

    return True


network_summary_second_table()



        0                    1       2       3     4       5       6
0    None  2018-01-02 00:00:00  Δ01/01  Δ12/26  None   Δ2017   Δ2016
1   fpRPM                76.72  0.1166 -0.0125  None  0.0351 -0.0876
2   fpPPC                70.7¢  0.1282  0.0158  None  0.0161 -0.0803
3    fpCY                0.109 -0.0102 -0.0278  None  0.0186 -0.2914
4  fpMLIY                 1.11  0.0528 -0.0001  None  0.1528  0.2887
5   fpVol                  9mm  0.3286  0.1989  None -0.2323  -0.093
6   fpRev              $0.65mm  0.4835  0.1839  None -0.2054 -0.1724
7     Rev              $3.18mm  0.4307   0.159  None -0.0474  0.0816
8     Vol                 55mm  0.2906  0.1873  None -0.1015  0.0917
9     RPM                57.68  0.1085 -0.0238  None  0.0593 -0.0103
Dates validated in Second Table
        0                    1       2       3     4       5       6
0    None  2018-01-02 00:00:00  Δ01/01  Δ12/26  None   Δ2017   Δ2016
1   fpRPM                76.72  0.1166 -0.0125  None  0.0351 -0.0876
2 

True

#### Check NW Summary Third Table (Smart Pricing)